In [19]:
# Dependencies
import tweepy
import numpy as np
import pandas as pd

# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Twitter API Keys
from config import (consumer_key,
                    consumer_secret,
                    access_token,
                    access_token_secret)

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [22]:
# Target Search Term
target_terms = ("@SouthwestAir", "@AmericanAir", "@SpiritAirlines",
                "@Virginatlantic", "@Delta", "@AlaskaAir", "@KLM", "@AskAirAsia") 


# "@JetBlue", "@SingaporeAir", "@ThaiAirways", "@AsianaAirlines", "@silkair", "@qatarairways", "@cathaypacific", "@lufthansa", 
# "@KoreanAir_KE", "@KLM", "@FlySWISS", "@Fly_Norwegian", "@Aeroflot_World", "@emirates", "@Qantas", "@FlyANA_official", 
# "@British_Airways", "@EL_AL_AIRLINES", "@SAS", "@EtihadAirways", "@CopaAirlines", "@AerLingus", "@RoyalJordanian", "@FlyAirNZ", 
# "@CzechAirlines", "@EVAAirUS", "@TurkishAirlines", "@porterairlines", "@CSAIRGlobal", "@aegeanairlines"

# "Real Person" Filters
min_tweets = 5
max_tweets = 10000
max_followers = 2500
max_following = 2500
lang = "en"

# List to hold results
results_list = []

# Loop through all target users
for target in target_terms:

    # Variable for holding the oldest tweet
    oldest_tweet = None

    # Variables for holding sentiments
    compound_list = []
    positive_list = []
    negative_list = []
    neutral_list = []

    # Loop through 10 times
    for x in range(10):

        # Run search around each tweet
        public_tweets = api.search(
            target, count=100, result_type="recent", max_id=oldest_tweet, wait_on_rate_limit=True)

        # Loop through all tweets
        for tweet in public_tweets["statuses"]:

            # Use filters to check if user meets conditions
            if (tweet["user"]["followers_count"] < max_followers
                and tweet["user"]["statuses_count"] > min_tweets
                and tweet["user"]["statuses_count"] < max_tweets
                and tweet["user"]["friends_count"] < max_following
                and tweet["user"]["lang"] == lang):

                # Run Vader Analysis on each tweet
                results = analyzer.polarity_scores(tweet["text"])
                compound = results["compound"]
                pos = results["pos"]
                neu = results["neu"]
                neg = results["neg"]

                # Add each value to the appropriate list
                compound_list.append(compound)
                positive_list.append(pos)
                negative_list.append(neg)
                neutral_list.append(neu)
                
            # Set the new oldest_tweet value
            oldest_tweet = tweet["id"] - 1

    # Store the Average Sentiments
    sentiment = {
        "User": target,
        "Compound": np.mean(compound_list),
        "Positive": np.mean(positive_list),
        "Neutral": np.mean(negative_list),
        "Negative": np.mean(neutral_list),
        "Tweet Count": len(compound_list)
    }

    # Print the Sentiments
    print(sentiment)
    print()
    
    # Append airline results to 'results_list'
    results_list.append(sentiment)

{'User': '@SouthwestAir', 'Compound': 0.15216102003642987, 'Positive': 0.13802914389799634, 'Neutral': 0.058284153005464474, 'Negative': 0.8036903460837888, 'Tweet Count': 549}

{'User': '@AmericanAir', 'Compound': 0.08545523465703972, 'Positive': 0.10899458483754514, 'Neutral': 0.07748916967509026, 'Negative': 0.813514440433213, 'Tweet Count': 554}

{'User': '@SpiritAirlines', 'Compound': -0.001141208791208791, 'Positive': 0.08520512820512821, 'Neutral': 0.08277106227106226, 'Negative': 0.832021978021978, 'Tweet Count': 546}

{'User': '@Virginatlantic', 'Compound': 0.26297248576850096, 'Positive': 0.14600379506641367, 'Neutral': 0.032390891840607214, 'Negative': 0.8216129032258065, 'Tweet Count': 527}

{'User': '@Delta', 'Compound': 0.21594291338582683, 'Positive': 0.11918897637795275, 'Neutral': 0.044013779527559055, 'Negative': 0.8367893700787401, 'Tweet Count': 508}

{'User': '@AlaskaAir', 'Compound': 0.2743133333333333, 'Positive': 0.14456068376068376, 'Neutral': 0.027046153846153

In [23]:
airline_df = pd.DataFrame(results_list).set_index("User").round(3)
airline_df

,Compound,Negative,Neutral,Positive,Tweet Count
User,,,,,
@SouthwestAir,0.152,0.804,0.058,0.138,549
@AmericanAir,0.085,0.814,0.077,0.109,554
@SpiritAirlines,-0.001,0.832,0.083,0.085,546
@Virginatlantic,0.263,0.822,0.032,0.146,527
@Delta,0.216,0.837,0.044,0.119,508
@AlaskaAir,0.274,0.828,0.027,0.145,585
@KLM,0.129,0.848,0.045,0.107,359
@AskAirAsia,-0.083,0.827,0.109,0.064,6
